# Milestone 1 Data Annotation User Interface
This notebook is designed to help you with the data annotation process for Milestone 1. Once you have collected preferences (i.e. A and B) for your questions, then you can load it with this notebook and rank your preferences visually. This notebook supports loading the questions and answers in two formats: Markdown and Latex. By default, it will display content in Markdown format. However, some questions might contain Latex code and hence, it could be easier to render these questions in Latex. To enable the Latex mode, please set the `LATEX_MODE` variable to `True` and run the UI cell again. 

## Setup

In [ ]:
# Install Python packages
!pip install ipykernel ipywidgets pdf2image

# Install Latex dependencies
# You need to install poppler and pdflatex libraries for Latex to work
# For pdflatex, check out this guide: https://www.latex-project.org/get/
# For poppler, you can install it with the following commands:
# On MacOS, if you are using Homebrew, you can install it with: brew install poppler
# On Ubuntu, you can install it with: sudo apt-get install poppler-utils
# On Windows, you can check out this guide (not tested by us): https://github.com/oschwartz10612/poppler-windows
# This is an optional tool, so you can change it as you wish and we will leave it to you to resolve any issues you may encounter.

## Annotation UI

In [ ]:
import os, json
import subprocess
from IPython.display import display, clear_output, Image, Markdown
import ipywidgets as widgets
from IPython.display import IFrame

DATAPATH = 'sample.json' # Path to your JSON file with A and B fields filled in
LATEX_DIR = "./latex_pdfs"
# Set to True to display LaTeX content, False to display Markdown content
# You need the pdf2image and poppler libraries installed for Latex to work
LATEX_MODE = False 

def load_data(datapath=DATAPATH):
    with open(datapath) as file:
        return json.load(file)

def save_data(data, datapath=DATAPATH):
    with open(datapath, 'w') as file:
        json.dump(data, file, indent=4)

data = load_data(DATAPATH)

current_index = 0

output = widgets.Output()

prev_button = widgets.Button(description="Previous", layout=widgets.Layout(width='100px'))
next_button = widgets.Button(description="Next", layout=widgets.Layout(width='100px'))

index_label = widgets.Label(value=f"Question {current_index + 1} of {len(data)}")

question_body_output = widgets.Output()
question_options_output = widgets.Output()
response_a_output = widgets.Output()
response_b_output = widgets.Output()

criteria_keys = ['overall', 'correctness', 'relevance', 'clarity', 'completeness']
criteria_buttons = {key: {} for key in criteria_keys}
button_functions = {}  

button_layout = widgets.Layout(width='200px')  

for key in criteria_keys:
    button_descriptions = ['A', 'B'] if key == 'overall' else ['A', 'B', 'AB']
    for value in button_descriptions:
        button = widgets.Button(description=value, button_style='info', layout=button_layout)
        criteria_buttons[key][value] = button
        button_functions[button] = key  

other_text = widgets.Text(description="Other", layout=widgets.Layout(width='500px'))

def compile_latex_to_pdf(latex_content, output_filename, directory=LATEX_DIR):
    """Compile LaTeX content to a PDF file."""
    if not os.path.exists(directory):
        os.makedirs(directory)

    tex_filename = os.path.join(directory, output_filename + '.tex')
    pdf_filename = os.path.join(directory, output_filename + '.pdf')
    
    # Full LaTeX document
    full_latex_document = r"""
\documentclass[10pt]{article}  % Set font size to 10pt
\usepackage{amsmath}
\usepackage[utf8]{inputenc}
\usepackage{amssymb}
\usepackage{hyperref}
\usepackage[margin=1cm]{geometry}  % Set margins to 0cm
\thispagestyle{empty}  % Avoid page numbers and headers/footers
\begin{document}
""" + latex_content + r"""
\end{document}
"""
    
    # Write LaTeX code to a .tex file
    with open(tex_filename, 'w') as file:
        file.write(full_latex_document)
    
    # Compile LaTeX to PDF
    latex_cmd = ['pdflatex', '-interaction=nonstopmode', '-output-directory', directory, tex_filename]
    result = subprocess.run(latex_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode != 0:
        return "Failed to compile LaTeX. Check the .tex file for errors."

    return pdf_filename

# Function to display criteria and setup buttons and text fields with markdown
def display_criteria_with_markdown(index):
    with output:
        clear_output(wait=True)
        question = data[index]
        with question_body_output:
            clear_output()
            display(Markdown("**Question body :** <br><br>" + question['question_body']) )
        if question.get('question_options'):
            options_md = "\n".join([f"- {option}" for option in question['question_options']])
            with question_options_output:
                clear_output()
                display(Markdown(options_md))
        else:
            with question_options_output:
                clear_output()
                display(Markdown(""))
        
        with response_a_output:
            clear_output()
            display(Markdown("<br>**A Response:** <br><br>" + question['A']))
        with response_b_output:
            clear_output()
            display(Markdown("<br>**B Response :**<br><br> " + question['B']))
        for key, buttons in criteria_buttons.items():
            selected_value = question['ranking_criteria'].get(key, '')
            for desc, button in buttons.items():
                if selected_value == desc:
                    button.button_style = 'success'
                else:
                    button.button_style = 'info'
                button.on_click(update_criteria)
        other_text.value = question['ranking_criteria'].get('other', '')
    index_label.value = f"Question {index + 1} of {len(data)}"

# Function to display criteria and setup buttons and text fields with latex
def display_criteria_with_latex(index):
    from pdf2image import convert_from_path
    
    with output:
        clear_output(wait=True)
        question = data[index]
        
        latex_content = (
            "\\section*{Question:}\n" + question['question_body'] + "\n\n" +
            "\\section*{Response A:}\n" + question['A'] + "\n\n" +
            "\\section*{Response B:}\n" + question['B'] + "\n\n"
        )
        
        pdf_filename = compile_latex_to_pdf(latex_content, f"combined_{index}")
        
        if isinstance(pdf_filename, str) and not pdf_filename.startswith("Failed"):
            images = convert_from_path(pdf_filename)
            
            for i, image in enumerate(images):
                image_path = f"{pdf_filename}_{i}.png"
                image.save(image_path)
                display(Image(filename=image_path))
                os.remove(image_path)  
                
        else:
            display("Error: " + pdf_filename)

        for key, buttons in criteria_buttons.items():
            selected_value = question['ranking_criteria'].get(key, '')
            for desc, button in buttons.items():
                if selected_value == desc:
                    button.button_style = 'success'
                else:
                    button.button_style = 'info'
                button.on_click(update_criteria)
        
        other_text.value = question['ranking_criteria'].get('other', '')
    index_label.value = f"Question {index + 1} of {len(data)}"

def display_criteria(index):
    if LATEX_MODE:
        display_criteria_with_latex(index)
    else:
        display_criteria_with_markdown(index)
      
def update_criteria(button):
    criterion_key = button_functions[button]  
    data[current_index]['ranking_criteria'][criterion_key] = button.description
    display_criteria(current_index)

def change_question(b):
    global current_index
    if b.description == "Next":
        data[current_index]['ranking_criteria']['other'] = other_text.value
        save_data(data)
        if current_index < len(data) - 1:
            current_index += 1
    elif b.description == "Previous":
        if current_index > 0:
            current_index -= 1
    display_criteria(current_index)

prev_button.on_click(change_question)
next_button.on_click(change_question)

navigation_layout = widgets.HBox([prev_button, index_label, next_button])
markdown_layout = widgets.VBox([question_body_output, question_options_output, response_a_output, response_b_output])
criteria_layouts = [widgets.HBox([widgets.Label(key.capitalize() + ':', layout=widgets.Layout(width='90px'))] + list(values.values())) for key, values in criteria_buttons.items()]
criteria_layouts.append(widgets.HBox([other_text]))

display_criteria(current_index)
display(navigation_layout, markdown_layout, widgets.VBox(criteria_layouts), output)